# Parallelism for Image Pre-processing

In [52]:
import numpy as np
import tensorflow as tf
import pickle
import cv2
import os
import matplotlib.pyplot as plt

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from numpy import asarray, save
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array



https://stackoverflow.com/questions/62759940/parallelization-strategies-for-deep-learning

In [2]:
import multiprocessing

In [50]:
# Set default image size
default_image_size = tuple((256, 256))

image_size = 0
directory_root = '../Data'
width=256
height=256
depth=3 

In [51]:
# function to convert images to array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size) 
          #  gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [53]:
image_list, label_list = [], []

try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)


    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        copy=listdir(f"{directory_root}/{plant_folder}")
        plant_list = []
        print(plant_disease_folder_list)
        for disease_folder in copy :
            # remove .DS_Store from list
            print(disease_folder)
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)
            
                
        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:150]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")


[INFO] Loading images ...
['Tomato_healthy', 'Potato___Early_blight', 'PlantVillage', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Early_blight', '.DS_Store', 'Tomato__Target_Spot', 'Potato___Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Septoria_leaf_spot', 'Tomato__Tomato_mosaic_virus', 'Pepper__bell___Bacterial_spot', 'Tomato_Bacterial_spot', 'Tomato_Late_blight', 'Pepper__bell___healthy', 'Potato___healthy']
Tomato_healthy
Potato___Early_blight
PlantVillage
Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato_Early_blight
.DS_Store
Tomato__Target_Spot
Potato___Late_blight
Tomato_Leaf_Mold
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato_Septoria_leaf_spot
Tomato__Tomato_mosaic_virus
Pepper__bell___Bacterial_spot
Tomato_Bacterial_spot
Tomato_Late_blight
Pepper__bell___healthy
Potato___healthy
[INFO] Processing Tomato_healthy ...
[INFO] Processing Potato___Early_blight ...
[INFO] Processing PlantVillage ...
[INFO] Processing Tomato__Tomato_Yel

In [55]:
#Binarize 
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

# Check number of classes
print(n_classes)

15


In [56]:
image_list = np.array(image_list, dtype=np.float32)

In [57]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [ ]:
for i in range(2250):


In [62]:
image_list

array([[[[102.,  88., 100.],
         [ 96.,  82.,  94.],
         [111.,  97., 109.],
         ...,
         [136., 122., 128.],
         [122., 108., 114.],
         [161., 147., 153.]],

        [[ 81.,  67.,  79.],
         [ 94.,  80.,  92.],
         [ 94.,  80.,  92.],
         ...,
         [140., 126., 132.],
         [140., 126., 132.],
         [132., 118., 124.]],

        [[107.,  93., 105.],
         [121., 107., 119.],
         [110.,  96., 108.],
         ...,
         [127., 113., 119.],
         [143., 129., 135.],
         [152., 138., 144.]],

        ...,

        [[151., 135., 142.],
         [ 84.,  68.,  75.],
         [150., 134., 141.],
         ...,
         [175., 158., 161.],
         [200., 183., 186.],
         [152., 135., 138.]],

        [[165., 149., 156.],
         [ 99.,  83.,  90.],
         [ 99.,  83.,  90.],
         ...,
         [173., 156., 159.],
         [176., 159., 162.],
         [124., 107., 110.]],

        [[ 70.,  54.,  61.],
       